In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
import os
import sys
sys.path.append('../..')

In [2]:
from data.dataloader import Covid19IndiaLoader

In [3]:
loader = Covid19IndiaLoader()
dataframes = loader.get_covid19india_api_data()

In [4]:
dataframes.keys()

dict_keys(['df_tested', 'df_statewise', 'df_india_time_series', 'df_districtwise', 'df_raw_data_old', 'df_raw_data', 'df_deaths_recoveries', 'df_districts', 'df_travel_history', 'df_resources'])

In [5]:
districtwise = dataframes['df_districtwise']
districtwise[districtwise['state'] == 'Delhi']['district'].unique()
# districtwise[districtwise['state'] == 'Gujarat']

array(['Central Delhi', 'East Delhi', 'New Delhi', 'North Delhi',
       'North East Delhi', 'North West Delhi', 'Shahdara', 'South Delhi',
       'South East Delhi', 'South West Delhi', 'West Delhi', 'Unknown'],
      dtype=object)

In [6]:
# dataframes['df_raw_data_2'].columns
# xyz = dataframes['df_raw_data_2']
# deceased = xyz[xyz['patientstatus'] == 'Deceased']
# # deceased[deceased['district'] == '']['state'].value_counts()
# state = deceased[deceased['state'] == 'Maharashtra']
# #  MANUAL CLEANUP + ASSUMPTIONS MADE IN CODE AFTER PRELIM MANUAL REVIEW
# state[state['district'] == ''].index[:20]
# state.loc[state[state['district'] == ''].index[:20]]
# # gj[gj['district'] == '']['district'] = 'Ahmadabad'

In [72]:
def get_district_death_df(df_deaths_recoveries, state, district):
    deceased = df_deaths_recoveries[df_deaths_recoveries['patientstatus'] == 'Deceased']
    statedf = deceased[deceased['state'] == state]
    unknown = statedf[statedf['district'] == '']
    print(f'{len(unknown)} deaths in {state} with unknown district')
    if state == 'Gujarat':
        print(f'adding {len(unknown)} deaths to Ahmadabad count')
        statedf['district'][statedf['district'] == ''] = 'Ahmadabad'
        districtdf = statedf[statedf['district'].isin(district)]
    elif state == 'Maharashtra':
        num_unknown = len(unknown)
        thirds = num_unknown//3
        print(f'adding {thirds} deaths to Mumbai/Pune each')
        print(len(statedf[statedf['district'].isin(district)]))
        unknown_index = statedf['district'][statedf['district'] == ''].index
        statedf.loc[statedf[statedf['district'] == ''].index[:thirds]] = 'Mumbai'
        statedf.loc[statedf[statedf['district'] == ''].index[thirds:]] = 'Pune'
        print(len(statedf[statedf['district'].isin(district)]))
        districtdf = statedf[statedf['district'].isin(district)]
    elif state == 'Rajasthan':
        districtdf = statedf[statedf['district'].isin(district)]
    elif state == 'Karnataka':
        districtdf = statedf[statedf['district'].isin(district)]
    elif state == 'Delhi':
        num_unknown = len(unknown)
        print(f'adding {num_unknown} deaths to New Delhi')
        statedf['district'][statedf['district'] == ''] = 'New Delhi'
        districtdf = statedf[statedf['district'].isin(district)]
    return districtdf

In [83]:
def get_district_time_series(dataframes, state='Karnataka', district='Bengaluru'):
    if district == 'all':
        districtwise = dataframes['df_districtwise']
        district = districtwise[districtwise['state'] == state]['district'].unique().tolist()
        district.append('')
    elif type(district) != list:
        district = [district]
        # district_timeseries = {}
        # for (s, d) in list(zip(state, district)):
        #     district_timeseries[d] = get_district_time_series(dataframes, state=s, district=d)
        # return district_timeseries
    # else:
    df_raw_data_1 = dataframes['df_raw_data'][dataframes['df_raw_data']['detectedstate'] == state]
    df_raw_data_1 = df_raw_data_1[df_raw_data_1['detecteddistrict'].isin(district)]
    if len(df_raw_data_1) == 0:
        return None
    df_raw_data_1['dateannounced'] = pd.to_datetime(df_raw_data_1['dateannounced'], format='%d/%m/%Y')
    index = pd.date_range(np.min(df_raw_data_1['dateannounced']), np.max(df_raw_data_1['dateannounced']))
    df_district = pd.DataFrame(columns=['total_confirmed'], index=index)
    df_district['total_confirmed'] = [0]*len(index)
    for _, row in df_raw_data_1.iterrows():
        df_district.loc[row['dateannounced']:, 'total_confirmed'] += 1

    # Deaths calculation
    deathsdf = get_district_death_df(dataframes['df_deaths_recoveries'], state, district)
    deathsdf = deathsdf[deathsdf['state'] == state]
    deathsdf = deathsdf[deathsdf['district'].isin(district)]
    deathsdf['date'] = pd.to_datetime(deathsdf['date'], format='%d/%m/%Y')
    df_district['total_deaths'] = [0]*len(index)
    
    for _, row in deathsdf.iterrows():
        if row['patientstatus'] == 'Deceased':
            date = pd.to_datetime(row['date'], format='%d/%m/%Y')
            df_district.loc[date:, 'total_deaths'] += 1

    df_district.reset_index(inplace=True)
    df_district.columns = ['date', 'total_confirmed', 'total_deaths']
    return df_district

In [86]:
# df_raw_data_1
district_timeseries = get_district_time_series(dataframes, state='Delhi', district='all')

['Central Delhi', 'East Delhi', 'New Delhi', 'North Delhi', 'North East Delhi', 'North West Delhi', 'Shahdara', 'South Delhi', 'South East Delhi', 'South West Delhi', 'West Delhi', 'Unknown', '']
54 deaths in Delhi with unknown district
adding 54 deaths to New Delhi
54
54


In [85]:
district_timeseries

,date,total_confirmed,total_deaths
0,2020-03-02,1,0
1,2020-03-03,1,0
2,2020-03-04,1,0
3,2020-03-05,2,0
4,2020-03-06,3,0
...,...,...,...
60,2020-05-01,2930,54
61,2020-05-02,2934,54
62,2020-05-03,2936,54
63,2020-05-04,2938,54


In [9]:
districts = ['Mumbai', 'Bengaluru', 'Ahmadabad', 'Jaipur', 'Pune', 'New Delhi']
district_timeseries = get_district_time_series(dataframes, state=['Maharashtra', 'Karnataka', 'Gujarat', 'Rajasthan', 'Maharashtra', 'Delhi'], district=districts)
district_timeseries.keys()

42 deaths in Maharashtra with unknown district
adding 14 deaths to Mumbai/Pune each
164
178
4 deaths in Karnataka with unknown district
6 deaths in Gujarat with unknown district
adding 6 deaths to Ahmadabad count
20 deaths in Rajasthan with unknown district
42 deaths in Maharashtra with unknown district
adding 14 deaths to Mumbai/Pune each
51
65
53 deaths in Delhi with unknown district


dict_keys(['Mumbai', 'Bengaluru', 'Ahmadabad', 'Jaipur', 'Pune', 'New Delhi'])

In [10]:
data = district_timeseries['Ahmadabad'].set_index('date')
data

,total_confirmed,total_deaths
date,,
2020-03-20,3,0
2020-03-21,5,0
2020-03-22,8,0
2020-03-23,14,0
2020-03-24,14,0
2020-03-25,14,0
2020-03-26,14,0
2020-03-27,14,0
2020-03-28,17,0


In [11]:
# get age data in bands
filenames = 'DDW-{}00C-13'
state_file_mapping = {
    filenames.format('24'): 'Gujarat',
    filenames.format('29'): 'Karnataka',
    filenames.format('27'): 'Maharashtra',
    filenames.format('07'): 'Delhi',
    filenames.format('08'): 'Rajasthan',
}

age_data = {}
directory = '../../data/data/census/'
for filename in os.listdir(directory):
    df = pd.read_excel(os.path.join(directory, filename))
    age_data[state_file_mapping[filename.split('.')[0]]] = df.dropna(how='all')

age_data.keys()

dict_keys(['Gujarat', 'Karnataka', 'Rajasthan', 'Maharashtra', 'Delhi'])

In [33]:
def clean(raw_all_district_age_data):
    transposed = raw_all_district_age_data.head(3).T
    transposed.fillna('', inplace=True)
    new = transposed[transposed.columns[0]]
    for x in transposed.columns[1:]:
        new += transposed[x] 
    all_district_age_data = raw_all_district_age_data.copy()
    all_district_age_data.columns = new.T
    return all_district_age_data[4:]

In [13]:
raw_all_district_age_data = age_data['Gujarat']


In [34]:
all_district_age_data = clean(raw_all_district_age_data)

In [39]:
test = clean(age_data['Karnataka'])
test['Area Name'].unique()

array(['State - KARNATAKA (29)', 'District - Belgaum (01)',
       'District - Bagalkot  (02)', 'District - Bijapur (03)',
       'District - Bidar (04)', 'District - Raichur (05)',
       'District - Koppal (06)', 'District - Gadag (07)',
       'District - Dharwad (08)', 'District - Uttara Kannada (09)',
       'District - Haveri (10)', 'District - Bellary (11)',
       'District - Chitradurga (12)', 'District - Davanagere (13)',
       'District - Shimoga (14)', 'District - Udupi (15)',
       'District - Chikmagalur (16)', 'District - Tumkur (17)',
       'District - Bangalore (18)', 'District - Mandya (19)',
       'District - Hassan (20)', 'District - Dakshina Kannada (21)',
       'District - Kodagu (22)', 'District - Mysore (23)',
       'District - Chamarajanagar (24)', 'District - Gulbarga (25)',
       'District - Yadgir (26)', 'District - Kolar (27)',
       'District - Chikkaballapura (28)',
       'District - Bangalore Rural (29)', 'District - Ramanagara (30)'],
      dty

In [15]:
# Get relevant district(s) data
district_age_data = all_district_age_data[all_district_age_data['Area Name'] == 'District - Ahmadabad (07)']
state_age_data = all_district_age_data[all_district_age_data['Area Name'] == 'State - GUJARAT (24)']
# or district_age_data['Distt.Code'] = 474 

In [16]:
# district_age_data['Age'].unique()

In [17]:
def create_age_bands():
    age_bands = {}
    for a in range(9):
        lower = a * 10
        upper = lower + 9
        if lower == 80:
            age_bands[f'{lower}+'] = list(range(lower, upper+11))
            age_bands[f'{lower}+'] += ['100+']
        else:
            age_bands[f'{lower}-{upper}'] = list(range(lower, upper+1))
    return age_bands

In [18]:
age_bands = create_age_bands()
age_bands

{'0-9': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 '10-19': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 '20-29': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 '30-39': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 '40-49': [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 '50-59': [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 '60-69': [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
 '70-79': [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
 '80+': [80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  '100+']}

In [19]:
def get_age_band_population(age_data):
    # filter for age bands and sum
    total_pop = age_data[age_data['Age'] == 'All ages']['TotalPersons'].values[0]
    unknown_pop = age_data[age_data['Age'] == 'Age not stated']['TotalPersons'].values[0]
    total_pop -= unknown_pop

    age_band_pops = {}
    for key, band in age_bands.items():
        pop = age_data[age_data['Age'].isin(band)]['TotalPersons'].sum()
        age_band_pops[key] = pop
    return age_band_pops, total_pop

In [20]:
state_age_band_pops, state_total_pop = get_age_band_population(state_age_data)
district_age_band_pops, district_total_pop = get_age_band_population(district_age_data)
district_age_band_pops, district_total_pop

({'0-9': 1216635,
  '10-19': 1348459,
  '20-29': 1387764,
  '30-39': 1144310,
  '40-49': 899585,
  '50-59': 620550,
  '60-69': 354147,
  '70-79': 158438,
  '80+': 65017},
 7194905)

In [21]:
assert(state_total_pop == sum(state_age_band_pops.values()))
assert(district_total_pop == sum(district_age_band_pops.values()))

In [22]:
state_age_band_ratios = {k: v / state_total_pop for k, v in state_age_band_pops.items()}
district_age_band_ratios = {k: v / district_total_pop for k, v in district_age_band_pops.items()}
district_age_band_ratios

{'0-9': 0.16909674276449793,
 '10-19': 0.18741859690989665,
 '20-29': 0.19288149044358474,
 '30-39': 0.15904449051099354,
 '40-49': 0.12503083779424468,
 '50-59': 0.08624853281593016,
 '60-69': 0.04922191467434247,
 '70-79': 0.022020860595101673,
 '80+': 0.009036533491408157}

In [23]:
ref_age_band_ratios = {
    '0-9': 0.1144545912,
    '10-19': 0.1096780507,
    '20-29': 0.1387701325,
    '30-39': 0.1481915984,
    '40-49': 0.1548679659,
    '50-59': 0.1428622446,
    '60-69': 0.1092853481,
    '70-79': 0.05542319854,
    '80+': 0.02646687006,
}

# calculated here: https://docs.google.com/spreadsheets/d/1APX7XwoJPIbUXOgXa2vZNreDn6UseBucSGq9fh-N5jE/edit#gid=1859974541
ref_mortality_rate = {
    '0-9': 0.0002447933091,
    '10-19': 0.000377136454,
    '20-29': 0.0009389093188,
    '30-39': 0.001769966645,
    '40-49': 0.003645033792,
    '50-59': 0.01343325563,
    '60-69': 0.03884219978,
    '70-79': 0.08406020728,
    '80+': 0.1421208672,
}

In [24]:
implied_mortality_age = {}
for k in ref_mortality_rate.keys():
    implied_mortality_age[k] = ref_age_band_ratios[k] * ref_mortality_rate[k]

implied_mortality = sum(implied_mortality_age.values())
# age weighted mortality based on other countries
implied_mortality

0.015610835492964907

In [25]:
# need mortality rate from India -- this is going to be dependent on case data, so introducing that testing bias...
daily_observed_mortality = data['total_deaths']/data['total_confirmed']
# observed deaths/known cases
daily_observed_mortality

date
2020-03-20    0.000000
2020-03-21    0.000000
2020-03-22    0.000000
2020-03-23    0.000000
2020-03-24    0.000000
2020-03-25    0.000000
2020-03-26    0.000000
2020-03-27    0.000000
2020-03-28    0.000000
2020-03-29    0.142857
2020-03-30    0.136364
2020-03-31    0.130435
2020-04-01    0.096774
2020-04-02    0.096774
2020-04-03    0.105263
2020-04-04    0.111111
2020-04-05    0.094340
2020-04-06    0.078125
2020-04-07    0.060241
2020-04-08    0.060241
2020-04-09    0.049296
2020-04-10    0.040609
2020-04-11    0.045267
2020-04-12    0.046099
2020-04-13    0.043750
2020-04-14    0.037534
2020-04-15    0.037778
2020-04-16    0.033028
2020-04-17    0.035313
2020-04-18    0.030162
2020-04-19    0.029973
2020-04-20    0.031250
2020-04-21    0.039330
2020-04-22    0.041972
2020-04-23    0.042373
2020-04-24    0.046129
dtype: float64

In [26]:
daily_mortality_ratio = daily_observed_mortality/implied_mortality
# how much different is it observed than 'implied'
daily_mortality_ratio

date
2020-03-20    0.000000
2020-03-21    0.000000
2020-03-22    0.000000
2020-03-23    0.000000
2020-03-24    0.000000
2020-03-25    0.000000
2020-03-26    0.000000
2020-03-27    0.000000
2020-03-28    0.000000
2020-03-29    9.151153
2020-03-30    8.735191
2020-03-31    8.355400
2020-04-01    6.199168
2020-04-02    6.199168
2020-04-03    6.742955
2020-04-04    7.117563
2020-04-05    6.043214
2020-04-06    5.004537
2020-04-07    3.858920
2020-04-08    3.858920
2020-04-09    3.157792
2020-04-10    2.601343
2020-04-11    2.899748
2020-04-12    2.953032
2020-04-13    2.802541
2020-04-14    2.404324
2020-04-15    2.419972
2020-04-16    2.115679
2020-04-17    2.262083
2020-04-18    1.932146
2020-04-19    1.919997
2020-04-20    2.001815
2020-04-21    2.519400
2020-04-22    2.688647
2020-04-23    2.714325
2020-04-24    2.954903
dtype: float64

In [27]:
age_stratified_daily_mortality = {}
for k in ref_mortality_rate.keys():
    age_stratified_daily_mortality[k] = daily_mortality_ratio * ref_mortality_rate[k]

# mortality rate accounting for observed/implied discrepancy
age_stratified_daily_mortality['70-79']

date
2020-03-20    0.000000
2020-03-21    0.000000
2020-03-22    0.000000
2020-03-23    0.000000
2020-03-24    0.000000
2020-03-25    0.000000
2020-03-26    0.000000
2020-03-27    0.000000
2020-03-28    0.000000
2020-03-29    0.769248
2020-03-30    0.734282
2020-03-31    0.702357
2020-04-01    0.521103
2020-04-02    0.521103
2020-04-03    0.566814
2020-04-04    0.598304
2020-04-05    0.507994
2020-04-06    0.420682
2020-04-07    0.324382
2020-04-08    0.324382
2020-04-09    0.265445
2020-04-10    0.218669
2020-04-11    0.243753
2020-04-12    0.248232
2020-04-13    0.235582
2020-04-14    0.202108
2020-04-15    0.203423
2020-04-16    0.177844
2020-04-17    0.190151
2020-04-18    0.162417
2020-04-19    0.161395
2020-04-20    0.168273
2020-04-21    0.211781
2020-04-22    0.226008
2020-04-23    0.228167
2020-04-24    0.248390
dtype: float64

In [28]:
age_std_mortality = sum([age_stratified_daily_mortality[k] * district_age_band_ratios[k] for k in district_age_band_ratios.keys()])
# mortality rate accounting for observed/implied discrepancy and weighted by model location age
age_std_mortality

date
2020-03-20    0.000000
2020-03-21    0.000000
2020-03-22    0.000000
2020-03-23    0.000000
2020-03-24    0.000000
2020-03-25    0.000000
2020-03-26    0.000000
2020-03-27    0.000000
2020-03-28    0.000000
2020-03-29    0.066220
2020-03-30    0.063210
2020-03-31    0.060462
2020-04-01    0.044859
2020-04-02    0.044859
2020-04-03    0.048794
2020-04-04    0.051505
2020-04-05    0.043730
2020-04-06    0.036214
2020-04-07    0.027924
2020-04-08    0.027924
2020-04-09    0.022851
2020-04-10    0.018824
2020-04-11    0.020983
2020-04-12    0.021369
2020-04-13    0.020280
2020-04-14    0.017398
2020-04-15    0.017512
2020-04-16    0.015310
2020-04-17    0.016369
2020-04-18    0.013982
2020-04-19    0.013894
2020-04-20    0.014486
2020-04-21    0.018231
2020-04-22    0.019456
2020-04-23    0.019642
2020-04-24    0.021382
dtype: float64

In [29]:
log_age_std_mortality = np.log(age_std_mortality)
log_age_std_mortality

date
2020-03-20        -inf
2020-03-21        -inf
2020-03-22        -inf
2020-03-23        -inf
2020-03-24        -inf
2020-03-25        -inf
2020-03-26        -inf
2020-03-27        -inf
2020-03-28        -inf
2020-03-29   -2.714770
2020-03-30   -2.761290
2020-03-31   -2.805742
2020-04-01   -3.104235
2020-04-02   -3.104235
2020-04-03   -3.020152
2020-04-04   -2.966085
2020-04-05   -3.129714
2020-04-06   -3.318305
2020-04-07   -3.578263
2020-04-08   -3.578263
2020-04-09   -3.778777
2020-04-10   -3.972622
2020-04-11   -3.864026
2020-04-12   -3.845818
2020-04-13   -3.898124
2020-04-14   -4.051381
2020-04-15   -4.044895
2020-04-16   -4.179274
2020-04-17   -4.112364
2020-04-18   -4.270019
2020-04-19   -4.276327
2020-04-20   -4.234596
2020-04-21   -4.004630
2020-04-22   -3.939612
2020-04-23   -3.930107
2020-04-24   -3.845185
dtype: float64

In [30]:
checker = pd.DataFrame()
print(district_total_pop)
checker['age_std'] = age_std_mortality
checker['non_std'] = data['total_deaths']/data['total_confirmed']
checker

7194905


,age_std,non_std
date,,
2020-03-20,0.000000,0.000000
2020-03-21,0.000000,0.000000
2020-03-22,0.000000,0.000000
2020-03-23,0.000000,0.000000
2020-03-24,0.000000,0.000000
2020-03-25,0.000000,0.000000
2020-03-26,0.000000,0.000000
2020-03-27,0.000000,0.000000
2020-03-28,0.000000,0.000000
